In [34]:
import os
import yaml
import json
from openai import OpenAI
import sqlite3
import pandas as pd
import time
import sqlglot

In [56]:
class LLDM_Assistant:

    class ItemNotFoundException(Exception):
        pass
    class ItemNotPossessedException(Exception):
        pass

    def __init__(self, api_key, db_name, excel_db_filename=None):
        self.client = OpenAI(api_key=api_key)
        if excel_db_filename:
            self.db = self.__create_db_from_file(excel_db_filename, db_name)
        else:
            self.db = self.__connect_to_existing_db(db_name)

        get_info_func_desc = '''
Generate a SQL query to extract information regarding items in a character's inventory, such as taking out or using an item the character possesses. Use the following view to generate the query, with column descriptions between asterisks (*):

CREATE TABLE IF NOT EXISTS CHARACTER_INVENTORY_DETAILS (
    Weapon_Name text, 
    Weapon_Description text,
    Total_Quantity int    
);

CREATE TABLE IF NOT EXISTS CHARACTER_INVENTORY_HISTORY_DETAILS (
    Weapon_Name text, 
    Weapon_Description text,
    Quantity int *A positive number represents obtained weapons, a negative number means discarded*,
    Modify_Time datetime
);

Query only existing columns from each table. Do not switch column-table identities.
Return only the SQL query without any preamble or post text, and without quotes. Do not add a semicolon at the end.
Only create SELECT queries and do not modify the table in any way.
    '''
        
        self.narrator = self.client.beta.assistants.create(
            name="narrator",
            instructions=
                """
                You are a DnD DM. You sets the scene by describing the environment and atmosphere, brings NPCs to life through detailed character portrayals, and narrates the outcomes of player actions. They establish the game's tone, provide world-building lore, guide the overarching story while balancing player choices, and enforce game rules.

                The information of the main character is as follows: Elara Windrider, a courageous warrior with a heart of gold, is a human fighter who embodies the principles of Lawful Good. She is tall and athletic, with short brown hair, green eyes, and a determined expression. Clad in chain mail and wielding a longsword, Elara's appearance reflects her readiness for battle. Born in a small village, she was trained by her father, a retired soldier. Driven by a desire to protect the innocent and seek justice, she left home to make her mark on the world. Elara is brave and compassionate, possessing a strong sense of justice. Though she is determined and reliable, her stubbornness can sometimes get the best of her.

                The plot summary is as follows:

                    The Dragon's Flagon (Tavern)
                Description: The Dragon's Flagon is a lively tavern with a warm, welcoming atmosphere. The walls are adorned with trophies from past adventurers, and a large fireplace dominates one side of the room.
                
                    Whispering Woods (Wilderness)
                Description: Whispering Woods is a foreboding forest with a canopy so thick it blocks out most of the sunlight. The air is filled with the sounds of unseen creatures, and the ground is covered with a thick layer of leaves.
                
                    Blackstone Keep
                Description: Blackstone Keep is a crumbling fortress with tall, dark towers and walls covered in ivy. Inside, it is dark and cold, with the air thick with the smell of decay.

                Prologue:
                In the tranquil town of Windshade, nestled between rolling hills and verdant forests, life moved at a gentle pace. The Dragon's Flagon tavern stood at the heart of this bustling village, a place where stories were shared, and adventures began. The tavern's warm glow and lively atmosphere provided a sanctuary for weary travelers and eager adventurers alike.
                Elara Windrider, a brave and determined adventurer, sat quietly at a corner table, listening intently to the tales of old. Trained by her father, a retired soldier, Elara had left her small village with a singular goal: to protect the innocent and uphold justice. Her heart yearned for a quest worthy of her skills and valor.
                On this fateful evening, an old traveler with a weathered face and a knowing smile shared a tale that caught Elara's attention. He spoke of someone in the Whispering Woods, a mysterious forest to the north, who knew the secret to defeating a fearsome dragon named Shadowflame. The dragon had taken residence in the abandoned Blackstone Keep, casting a shadow of terror over the region.
                Determined and inspired by the traveler's tale, Elara, despite having no gear or weapon, prepared herself and set off into the wilderness, ready to face the unknown and embrace her destiny. Now, she walks into the heart of the Whispering Woods, her journey already begun.

                After receiving user response, you generate a narrative that moves the plot forward while maintaining a realistic continuity of events.
                """,

            tools=[
                # {"type": "file_search"},
                {
                    "type": "function",
                    "function": {
                        "name": "get_obtained_item",
                        "description": "Extract the item that the user has obtained in some manner (such as picked up, purchased, etc.)",
                        "parameters": {
                            "type": "object",
                            "properties": {
                                "item_name": {
                                    "type": "string",
                                    "description": "The name of the obtained item.",
                                },
                                "quantity": {
                                    "type": "integer",
                                    "description": "The number of said items obtained. If a number is not specified, try and infer based on the surrounding context."
                                }
                            },
                            "required": ["item_name", "quantity"],
                        },
                    },
                },
                {
                    "type": "function",
                    "function": {
                        "name": "get_discarded_item",
                        "description": "Extract the item that the user has discarded in some manner (such as thrown away, consumed, broken, etc.)",
                        "parameters": {
                            "type": "object",
                            "properties": {
                                "item_name": {
                                    "type": "string",
                                    "description": "The name of the discarded item.",
                                },
                                "quantity": {
                                    "type": "integer",
                                    "description": "The number of said items discarded. If a number is not specified, try and infer based on the surrounding context."
                                }
                            },
                            "required": ["item_name", "quantity"],
                        },
                    },
                },
                {
                    "type": "function",
                    "function": {
                        "name": "get_item_info",
                        "description": get_info_func_desc,
                        "parameters": {
                            "type": "object",
                            "properties": {
                                "sql_query": {
                                    "type": "string",
                                    "description": "The generated SQL query that would extract the information related to the character inventory requested by the user",
                                }
                            },
                            "required": ["sql_query"],
                        },
                    },
                }
            ],
            model="gpt-3.5-turbo",
        )

        self.thread_narrator = self.client.beta.threads.create()

    def narrator_chat(self, content):
        message = self.client.beta.threads.messages.create(
            thread_id=self.thread_narrator.id,
            role="user",
            content=f"""
            {content}
            """,
        )

        run = self.client.beta.threads.runs.create_and_poll(
            thread_id=self.thread_narrator.id,
            assistant_id=self.narrator.id,
        )

        counter = 0
        while run.status != 'completed':
            print(run.status)
            if run.status == 'requires_action':
                ###print('Function calling...')
                required_actions = run.required_action.submit_tool_outputs.model_dump()
                tool_outputs = []
                ###print(required_actions["tool_calls"])
                for action in required_actions["tool_calls"]:
                    func_name = action['function']['name']
                    arguments = json.loads(action['function']['arguments'])
                    available_functions = {
                        "get_obtained_item": self.__get_obtained_item,
                        "get_discarded_item": self.__get_discarded_item,
                        'get_item_info': self.__get_item_info
                    }
                    try:
                        function_to_call = available_functions[func_name]
                        # print(arguments)
                        # function_args = json.loads(arguments)
                        # output = function_to_call(
                        #     item_name=function_args.get("item_name"),
                        #     quantity=function_args.get("quantity"),
                        # )
                        output = function_to_call(
                            **arguments
                        )
                        output_string = json.dumps(output)
                        ###print(output_string)
                        tool_outputs.append({
                            "tool_call_id": action['id'],
                            "output": output_string
                        })
                    except: # function not found
                        raise ValueError(f"Unknown function: {func_name}")
                    
                ###print("Submitting outputs back to the Assistant…")
                ###print(run.status)
                run = self.client.beta.threads.runs.submit_tool_outputs(
                    thread_id=self.thread_narrator.id,
                    run_id=run.id,
                    tool_outputs=tool_outputs
                )
                # time.sleep(5) 
            if run.status == "cancelled":
                print("Run cancelled.")
                break
            if run.status == "cancelling":
                print("Run cancelling.")
                break
            if run.status == "failed":
                print("Run failed.")
                break
            if run.status == "expired":
                print("Run expired.")
                break
            else:            
                run = self.client.beta.threads.runs.retrieve(
                    thread_id=self.thread_narrator.id,
                    run_id=run.id,
                )
                time.sleep(5)  
                # if counter % 10 == 0:
                #     # print('Waiting for assistant. Status:', run.status)
                #     counter += 1
                #     time.sleep(5)  
                
            
        
        messages_narrator = self.client.beta.threads.messages.list(
            thread_id=self.thread_narrator.id
        )
        chat_history_narrator = []
        for thread_message in messages_narrator.data:
            for content_item in thread_message.content:
                role = thread_message.role
                id = thread_message.assistant_id
                item = content_item.text.value
                chat_history_narrator.append({'role': role, 'content': item})

        return chat_history_narrator

    def get_inventory_snapshot(self):
        query = '''
        SELECT * FROM CHARACTER_INVENTORY_DETAILS;
        '''
        return self.__run_query(query)

    # connect to db without erasing
    # used for continuous web app
    def __connect_to_existing_db(self, db_name):
        try: 
            db = sqlite3.connect(db_name, check_same_thread=False) 
            print(f"Database {db_name} connected.") 
        except Exception as e: 
            print(f"Database {db_name} not connected.")
            print(repr(e))
        
        return db

    # used to create fresh db
    def __create_db_from_file(self, excel_filename, db_name):
        try: 
            db = sqlite3.connect(db_name, check_same_thread=False) 
            print(f"Database {db_name} formed.") 
        except: 
            print(f"Database {db_name} not formed.")

        # erase current db contents
        cursor = db.cursor()
        query = '''
        PRAGMA writable_schema = 1;
        DELETE FROM sqlite_master;
        PRAGMA writable_schema = 0;
        VACUUM;
        PRAGMA integrity_check;
        '''
        cursor.executescript(query)

        # create db from excel spreadsheet
        dfs = pd.read_excel(excel_filename, sheet_name=None)
        for table, df in dfs.items():
            try:
                print(table.upper().replace(' ','_').strip())
                df.columns = [x.replace(' ','_') for x in df.columns.values]
                df.to_sql(table.upper().replace(' ','_').strip(), db)
            except:
                continue

        # modify columns
        query = '''
        CREATE TABLE IF NOT EXISTS CAMPAIGN (
            Campaign_ID INTEGER NOT NULL,
            Setting TEXT NOT NULL,
            Start_Time TEXT NOT NULL,
            Current_Turns INTEGER NOT NULL DEFAULT 0
        );

        ALTER TABLE CHARACTER_SHEET
        RENAME COLUMN "index" TO Character_ID;

        ALTER TABLE CHARACTER_SHEET
        ADD COLUMN Campaign_ID INTEGER;

        ALTER TABLE INVENTORY
        RENAME COLUMN "index" TO Item_ID;

        ALTER TABLE WORLD_ITEMS
        RENAME COLUMN "index" TO Item_ID;

        ALTER TABLE SETTINGS
        RENAME COLUMN "index" TO Setting_ID;

        ALTER TABLE NPCS
        RENAME COLUMN "index" TO NPC_ID;

        ALTER TABLE TREASURES
        RENAME COLUMN "index" TO Treasure_ID;

        ALTER TABLE MONSTERS
        RENAME COLUMN "index" TO Monster_ID;

        ALTER TABLE PLOT
        RENAME COLUMN "index" TO Plot_ID;

        ALTER TABLE PLOT
        ADD COLUMN Campaign_ID INTEGER;

        ALTER TABLE LOGS
        RENAME COLUMN "index" TO Log_ID;

        ALTER TABLE LOGS
        ADD COLUMN Campaign_ID INTEGER;

        ALTER TABLE LOGS
        ADD COLUMN Character_ID INTEGER;

        CREATE TABLE IF NOT EXISTS CHARACTER_INVENTORY (
            Campaign_ID INTEGER NOT NULL,
            Character_ID INTEGER NOT NULL,
            Item_ID INTEGER NOT NULL,
            Total_Quantity FLOAT DEFAULT 0
        );

        CREATE TABLE IF NOT EXISTS CHARACTER_INVENTORY_HISTORY (
            Campaign_ID INTEGER NOT NULL,
            Character_ID INTEGER NOT NULL,
            Item_ID INTEGER NOT NULL,
            Quantity FLOAT DEFAULT 0,
            Modify_Time DATETIME DEFAULT CURRENT_TIMESTAMP
        );

        CREATE VIEW IF NOT EXISTS CHARACTER_INVENTORY_DETAILS 
        AS
        SELECT
            a.Campaign_ID, a.Character_ID, a.Item_ID, a.Total_Quantity,
            b.Weapon_Name, b.Weapon_Description
        FROM CHARACTER_INVENTORY a
        JOIN WORLD_ITEMS b ON a.Item_ID = b.Item_ID;

        CREATE VIEW IF NOT EXISTS CHARACTER_INVENTORY_HISTORY_DETAILS 
        AS
        SELECT
            a.Campaign_ID, a.Character_ID, a.Item_ID, a.Quantity, a.Modify_Time,
            b.Weapon_Name, b.Weapon_Description
        FROM CHARACTER_INVENTORY_HISTORY a
        JOIN WORLD_ITEMS b ON a.Item_ID = b.Item_ID;
        '''
        cursor.executescript(query)
        db.commit()
        return db
        

    def __run_query(self, query):
        try:
            df = pd.read_sql_query(query, self.db)
            return df
        # not a select statement
        except TypeError:
            cursor = self.db.cursor()
            cursor.execute(query)
            return  
        
    def __validate_item(self, item_name):
        query = f'''
        SELECT * FROM WORLD_ITEMS WHERE UPPER(Weapon_Name) LIKE "{item_name.upper()}%" 
        '''
        tmp = self.__run_query(query)
        if not tmp.empty:
            return tmp['Item_ID'].iloc[0]
        return None

    # update table if item validated, otherwise error message
    # for now, use temporary campaign and character id
    def __get_obtained_item(self, item_name, quantity, campaign_id=0, character_id=0):
        item_id = self.__validate_item(item_name)
        print(item_id, quantity)
        if item_id is not None:
            # TODO: error handling
            cursor = self.db.cursor()

            # overall character inventory tracker update
            query = f'''
            UPDATE CHARACTER_INVENTORY SET Total_Quantity = Total_Quantity + {quantity}
            WHERE Item_ID = {item_id} AND Campaign_ID = {campaign_id} AND Character_ID = {character_id}
            '''
            cursor.execute(query)

            if cursor.rowcount == 0:
                query = f'''
                INSERT INTO CHARACTER_INVENTORY (Campaign_ID, Character_ID, Item_ID, Total_Quantity) VALUES ({campaign_id}, {character_id}, {item_id}, {quantity})
                '''
                cursor.execute(query)

            # inventory history update
            query = f'''
            INSERT INTO CHARACTER_INVENTORY_HISTORY (Campaign_ID, Character_ID, Item_ID, Quantity) VALUES ({campaign_id}, {character_id}, {item_id}, {quantity})
            '''
            cursor.execute(query)

            return json.dumps({'message':'The item(s) were successfully obtained. Please continue the story.'})
        else:
            return json.dumps({'message':'Item does not exist. Please prompt user to specify further or provide another action.'})

    # check if item queried is in INVENTORY table and character has more than discard amount
    # might need a third error condition if item exists, user has item, but has less than discard amount
    def __validate_item_discard(self, item_name, quantity, campaign_id, character_id):
        query = f'''
        SELECT * FROM WORLD_ITEMS WHERE UPPER(Weapon_Name) LIKE "{item_name.upper()}%" 
        '''
        tmp = self.__run_query(query)
        if not tmp.empty: # item exists in world
            item_id = tmp['Item_ID'].iloc[0]
            # validate if character has item
            query = f'''
            SELECT * FROM CHARACTER_INVENTORY 
            WHERE Campaign_ID = {campaign_id} AND Character_ID = {character_id} AND Item_ID = {item_id} AND Total_Quantity >= {quantity}
            '''
            # there should theoretically only be one row of the item for each character with the quantity as different values
            # need to validate and put checks in place
            tmp = self.__run_query(query)
            if not tmp.empty:
                return item_id
            raise self.ItemNotPossessedException("Character does not have the item in their inventory.")
        raise self.ItemNotFoundException("Item does not exist in this campaign.")

    # update table if item validated, otherwise error message
    # for now, use temporary campaign and character id
    # first update with item subtraction, then remove all entries with <=0 values
    def __get_discarded_item(self, item_name, quantity, campaign_id=0, character_id=0):
        try:
            item_id = self.__validate_item_discard(item_name, quantity, campaign_id, character_id)
            
            cursor = self.db.cursor()

            # overall inventory tracker update
            query = f'''
            UPDATE CHARACTER_INVENTORY SET Total_Quantity = Total_Quantity - {quantity}
            WHERE Item_ID = {item_id} AND Campaign_ID = {campaign_id} AND Character_ID = {character_id};
            '''
            cursor.execute(query)

            # housekeeping query, remove rows with invalid quantites (<= 0 items)
            query = 'DELETE FROM CHARACTER_INVENTORY WHERE Total_Quantity <= 0'
            cursor.execute(query)

            # inventory history tracker update
            # use negative quantity value to indicate 
            query = f'''
            INSERT INTO CHARACTER_INVENTORY_HISTORY (Campaign_ID, Character_ID, Item_ID, Quantity) VALUES ({campaign_id}, {character_id}, {item_id}, {-quantity})
            '''

            return json.dumps({'message':'The item(s) were successfully discarded. Please continue the story.'})
        except self.ItemNotPossessedException as e:
            return json.dumps({'message':"Item is not in character's possession. Please prompt user to specify further or provide another action."})
        except self.ItemNotFoundException as e:
            return json.dumps({'message':"Item does not exist. Please prompt user to specify further or provide another action."})


    def __get_item_info(self, sql_query, campaign_id=0, character_id=0):
        self.__run_query('PRAGMA QUERY_ONLY = ON;')
        try:
            
            # at some point in life, will figure out how to remove existing ID matching clauses before adding these
            # but for now this will do
            where = sqlglot.condition(f'Campaign_ID={campaign_id}').and_(f'Character_ID={character_id}')
            sql_query = sqlglot.parse_one(sql_query).where(where).sql()
            ###print(sql_query)

            df_result = self.__run_query(sql_query)
            result = json.dumps(df_result.to_dict())
            print(result)

            self.__run_query('PRAGMA QUERY_ONLY = OFF;')
            return json.dumps({'message':f"The result of the user's request in JSON format is {result}. Please use this to answer the user's question or honor the user's request."})
        except Exception as e:
            self.__run_query('PRAGMA QUERY_ONLY = OFF;')
            return json.dumps({'message':"Something went wrong, please prompt the user for another action"})
        

In [57]:
import os
current_directory = os.getcwd()
print(current_directory)

# Path to your Excel file
file_path = 'dnd_weapon_prompts.xlsx'

# Read the Excel file
df = pd.read_excel(file_path)
df.head(100)
OPENAI_API_KEY = 
client = OpenAI(api_key=OPENAI_API_KEY)

/Users/wudiqing/w210_capstone


In [58]:
# Harder Mode
numRound = 20
numIter = 40
maxFailed = 10
# List of weapon names
weapons = list(df["Weapon Name"])

# Generating sentences
#sentences = [f"Elara notices a treasure chest in the corner and opens it. There is a {weapon} in it and Elara picks it up, feeling its power surge through her. She likes it and puts {weapon} in her weapon inventory." for weapon in weapons]
sentences = [f"Elara picks the {weapon} up, feeling its power surge through her. Then she puts it in her weapon inventory." for weapon in weapons]
df['Prompt_Pickup'] = sentences

sentences = [f"Elara wanders in the wilderness and starts to feel that she is carrying too many items with her. She checks through her inventory, takes out {weapon}, and discard it on the ground, before continuing her adventure." for weapon in weapons]
df['Prompt_Drop'] = sentences

In [68]:
# Harder Mode
# pick-up only
# do i need to initiate the narrator every round?
excel_db_filename = 'DnD.xlsx'
db_name = "lldmDWU_HM.db"

for j in range(51, 60):
    lldm_assistant = LLDM_Assistant(OPENAI_API_KEY, db_name)
    failed_count = 0
    evaluationHowManyPicked = []
    evaluationPickedOrNot_ShouldBeNo = []
    evaluationPickedOrNot_ShouldBeYes = []
    evaluationPickedMostRecent = []
    print(f"Evaluation Round {j} Starts...")
    for i in range(numIter):
        print(f"Iteration {i} Starts...")
        if (i%5 == 1) | (i%5 == 2):
            # pick up an item
            idxEval = i
            tmp = lldm_assistant.narrator_chat(df['Prompt_Pickup'][idxEval])
            response = tmp[0]['content']
            while (response is None) & (failed_count < maxFailed):
                tmp = lldm_assistant.narrator_chat(df['Prompt_Pickup'][idxEval])
                response = tmp[0]['content']
                failed_count+=1
                print(f"failed count: {failed_count}")
            display(lldm_assistant.get_inventory_snapshot())
        elif i%5 == 4:
            # question 1: how many
            tmp = lldm_assistant.narrator_chat("How many weapon or armor does Elara have?")
            response = tmp[0]['content']
            while (response is None) & (failed_count < maxFailed):
                tmp = lldm_assistant.narrator_chat("How many weapon or armor does Elara have?")
                response = tmp[0]['content']
                failed_count+=1
                print(f"failed count: {failed_count}")
            evaluationHowManyPicked.append([i, response])
            time.sleep(1)
            
            
            # question 2: has picked or not? Yes
            weaponEval = df['Weapon Name'][idxEval]
            tmp = lldm_assistant.narrator_chat("Have I picked up " + weaponEval + " before?")
            response = tmp[0]['content']
            while (response is None) & (failed_count < maxFailed):
                tmp = lldm_assistant.narrator_chat("Have I picked up " + weaponEval + " before?")
                response = tmp[0]['content']
                failed_count+=1
                print(f"failed count: {failed_count}")
            evaluationPickedOrNot_ShouldBeYes.append([i, weaponEval, response])
            time.sleep(1)
                      
            # question 3: has picked or not? No
            if idxEval+2<numIter:
                weaponEval = df['Weapon Name'][idxEval+2]          
                tmp = lldm_assistant.narrator_chat("Have I picked up " + weaponEval + " before?")
                response = tmp[0]['content']
                while (response is None) & (failed_count < maxFailed):
                    tmp = lldm_assistant.narrator_chat("Has Elara picked up " + weaponEval + " before?")
                    response = tmp[0]['content']
                    failed_count+=1
                    print(f"failed count: {failed_count}")
                evaluationPickedOrNot_ShouldBeNo.append([i, weaponEval, response])
            time.sleep(1)
            
            
            # question 4: latest
            tmp = lldm_assistant.narrator_chat("What was the latest weapon or armor I obtained?")
            weaponEval = df['Weapon Name'][idxEval]
            response = tmp[0]['content']
            while (response is None) & (failed_count < maxFailed):
                tmp = lldm_assistant.narrator_chat("What was the latest weapon or armor I obtained?")
                response = tmp[0]['content']
                failed_count+=1
                print(f"failed count: {failed_count}")
            evaluationPickedMostRecent.append([i, weaponEval, response])
        else:
            # story telling prompt
            tmp = lldm_assistant.narrator_chat("Elara feels tired and decides to find a place to go to sleep. The night is peaceful and quiet. She wakes up the next morning and continues her adventure.")
            response = tmp[0]['content']
            while (response is None) & (failed_count < maxFailed):
                tmp = lldm_assistant.narrator_chat("Elara feels tired and decides to find a place to go to sleep. The night is peaceful and quiet. She wakes up the next morning and continues her adventure.")
                response = tmp[0]['content']
                failed_count+=1
                print(f"failed count: {failed_count}") 
        
        print(tmp[1]['content'].strip())
        print(response)
        time.sleep(10)      
        df_eval1 = pd.DataFrame(evaluationHowManyPicked, columns=['iterations', 'answer'])
        df_eval2 = pd.DataFrame(evaluationPickedOrNot_ShouldBeNo, columns=['iterations', 'item', 'answer'])
        df_eval3 = pd.DataFrame(evaluationPickedOrNot_ShouldBeYes, columns=['iterations', 'item', 'answer'])
        df_eval4 = pd.DataFrame(evaluationPickedMostRecent, columns=['iterations', 'item', 'answer'])

        df_eval1.to_excel(f"{current_directory}/evaluation_llm4llm/HM_pickupOnly_evalHowMany_{j}.xlsx")
        df_eval2.to_excel(f"{current_directory}/evaluation_llm4llm/HM_pickupOnly_evalPickedOrNot_No_{j}.xlsx")
        df_eval3.to_excel(f"{current_directory}/evaluation_llm4llm/HM_pickupOnly_evalPickedOrNot_Yes_{j}.xlsx")
        df_eval4.to_excel(f"{current_directory}/evaluation_llm4llm/HM_pickupOnly_evalMostRecent_{j}.xlsx")
        
    time.sleep(10)
    display(lldm_assistant.get_inventory_snapshot())
        
        
    # delete the threads at the end of each round
    #client.beta.threads.delete(thread_narrator.id)
    print(f"Evaluation Round {j} Threads Deleted.")
    #create_narrator()                  

Database lldm.db connected.
Evaluation Round 51 Starts...
Iteration 0 Starts...
Elara feels tired and decides to find a place to go to sleep. The night is peaceful and quiet. She wakes up the next morning and continues her adventure.
As the night falls in the Whispering Woods, Elara, feeling the weight of the day's journey, seeks out a safe spot to rest. Finding a secluded clearing beneath a towering oak tree, she lays her bedroll and settles in for the night, the sounds of the forest lulling her into a deep slumber.

The night passes peacefully, undisturbed by any untoward events. As the first light of dawn filters through the thick canopy above, Elara awakens, refreshed and ready to continue her quest. The air is cool and crisp, filled with the earthy scents of the forest.

After a quick breakfast of dried fruits and jerky, Elara packs up her gear and sets off deeper into the Whispering Woods, her heart filled with determination and purpose. The path ahead is shrouded in mist, and th

,Campaign_ID,Character_ID,Item_ID,Total_Quantity,Weapon_Name,Weapon_Description
0,0,0,1,1.0,Forgotten Lance of Knight,The Forgotten Lance of Knight is a lance known...


Elara picks the Forgotten Lance of Knight up, feeling its power surge through her. Then she puts it in her weapon inventory.
As Elara ventures deeper into the Whispering Woods, her keen eyes catch a glimmer of light amidst the undergrowth. Kneeling down, she uncovers a forgotten relic: the legendary Forgotten Lance of Knight. As she grasps the lance, a surge of power courses through her, and she can feel the ancient energies bound within it.

With reverence, Elara carefully stows the lance in her weapon inventory, feeling a newfound sense of strength and purpose. The weight of the lance in her hands serves as a reminder of the noble knights who once wielded it, instilling her with determination to carry on their legacy.

Armed with the Forgotten Lance of Knight, Elara continues her journey through the Whispering Woods, her heart brimming with valor and her resolve unshaken. The forest around her seems to whisper in awe of the warrior who now bears the legendary weapon, her path illumin

,Campaign_ID,Character_ID,Item_ID,Total_Quantity,Weapon_Name,Weapon_Description
0,0,0,1,1.0,Forgotten Lance of Knight,The Forgotten Lance of Knight is a lance known...
1,0,0,2,1.0,Cursed Armor of Warrior,The Cursed Armor of Warrior is a armor known t...


Elara picks the Cursed Armor of Warrior up, feeling its power surge through her. Then she puts it in her weapon inventory.
As Elara moves further into the heart of the Whispering Woods, her sharp eyes catch another glint of light amidst the shadows. She uncovers a set of ancient armor, its steel imbued with an ominous aura—the Cursed Armor of Warrior. Despite the foreboding aura surrounding it, Elara feels a surge of power as she dons the armor, each piece fitting her like a second skin.

The weight of the cursed armor is palpable, and yet Elara stands taller, her resolve unwavering as she embraces the newfound strength it bestows upon her. The whispers of the forest seem to grow faint in the presence of the armor's dark enchantment, but Elara's determination remains steadfast.

With a solemn expression, Elara places the Cursed Armor of Warrior in her weapon inventory, a silent promise to wield its power with caution and purpose. The armor's presence serves as a reminder of the trials 

...

Elara picks the Enchanted Axe of Frost up, feeling its power surge through her. Then she puts it in her weapon inventory.
The Enchanted Axe of Frost joins Elara's collection of legendary artifacts as she discovers its powerful icy enchantments within the Whispering Woods. The surge of frosty energy that courses through her serves as a reminder of the elemental forces at her disposal.

With the Enchanted Axe of Frost securely stored in her weapon inventory, Elara's resolve deepens as she prepares to face the challenges that lie ahead. The chilling touch of the axe imbues her with a newfound strength and determination as she continues her quest for justice and valor through the mystical forest.
Iteration 37 Starts...
requires_action
37 1
in_progress


,Campaign_ID,Character_ID,Item_ID,Total_Quantity,Weapon_Name,Weapon_Description
0,0,0,1,1.0,Forgotten Lance of Knight,The Forgotten Lance of Knight is a lance known...
1,0,0,2,1.0,Cursed Armor of Warrior,The Cursed Armor of Warrior is a armor known t...
2,0,0,6,1.0,Mystic Axe of Ranger,The Mystic Axe of Ranger is a axe known throug...
3,0,0,7,1.0,Forgotten Wand of Frost,The Forgotten Wand of Frost is a wand known th...
4,0,0,11,1.0,Shadow Armor of Knight,The Shadow Armor of Knight is a armor known th...
5,0,0,12,1.0,Cursed Sword of Ranger,The Cursed Sword of Ranger is a sword known th...
6,0,0,16,1.0,Ancient Bow of Flame,The Ancient Bow of Flame is a bow known throug...
7,0,0,17,1.0,Divine Wand of Assassin,The Divine Wand of Assassin is a wand known th...
8,0,0,21,1.0,Forgotten Helmet of Mage,The Forgotten Helmet of Mage is a helmet known...
9,0,0,22,1.0,Legendary Ring of Frost,The Legendary Ring of Frost is a ring known th...


Elara picks the Mystic Ring of Mage up, feeling its power surge through her. Then she puts it in her weapon inventory.
The Mystic Ring of Mage graces Elara's possession, its arcane power coursing through her veins as she adds the magical artifact to her growing inventory. With each new acquisition, her connection to the mystical energies of the Whispering Woods deepens, further augmenting her prowess on her quest for justice and valor.
Iteration 38 Starts...
Elara feels tired and decides to find a place to go to sleep. The night is peaceful and quiet. She wakes up the next morning and continues her adventure.
As night falls over the Whispering Woods, Elara seeks a tranquil grove bathed in the soft glow of the moon to rest. The ancient trees stand sentinel as she settles down, her enchanted artifacts a comforting presence by her side. Dreams of her noble quest and the challenges ahead dance through her mind, melding with the whispers of the woodland spirits.

With the first light of daw

,Campaign_ID,Character_ID,Item_ID,Total_Quantity,Weapon_Name,Weapon_Description
0,0,0,1,1.0,Forgotten Lance of Knight,The Forgotten Lance of Knight is a lance known...
1,0,0,2,1.0,Cursed Armor of Warrior,The Cursed Armor of Warrior is a armor known t...
2,0,0,6,1.0,Mystic Axe of Ranger,The Mystic Axe of Ranger is a axe known throug...
3,0,0,7,1.0,Forgotten Wand of Frost,The Forgotten Wand of Frost is a wand known th...
4,0,0,11,1.0,Shadow Armor of Knight,The Shadow Armor of Knight is a armor known th...
5,0,0,12,1.0,Cursed Sword of Ranger,The Cursed Sword of Ranger is a sword known th...
6,0,0,16,1.0,Ancient Bow of Flame,The Ancient Bow of Flame is a bow known throug...
7,0,0,17,1.0,Divine Wand of Assassin,The Divine Wand of Assassin is a wand known th...
8,0,0,21,1.0,Forgotten Helmet of Mage,The Forgotten Helmet of Mage is a helmet known...
9,0,0,22,1.0,Legendary Ring of Frost,The Legendary Ring of Frost is a ring known th...


Evaluation Round 59 Threads Deleted.


# with discard

In [75]:
import os
import numpy as np
current_directory = os.getcwd()
print(current_directory)

# Path to your Excel file
file_path = 'dnd_weapon_prompts.xlsx'

# Read the Excel file
df = pd.read_excel(file_path)
df.head(100)
OPENAI_API_KEY = 
client = OpenAI(api_key=OPENAI_API_KEY)

/Users/wudiqing/w210_capstone


In [76]:
# Harder Mode
numRound = 30
numIter = 40
maxFailed = 15
# List of weapon names
weapons = list(df["Weapon Name"])

# Generating sentences
#sentences = [f"Elara notices a treasure chest in the corner and opens it. There is a {weapon} in it and Elara picks it up, feeling its power surge through her. She likes it and puts {weapon} in her weapon inventory." for weapon in weapons]
sentences = [f"Elara picks the {weapon} up, feeling its power surge through her. Then she puts it in her weapon inventory." for weapon in weapons]
df['Prompt_Pickup'] = sentences

sentences = [f"Elara wanders in the wilderness and starts to feel that she is carrying too many items with her. She checks through her inventory, takes out {weapon}, and discard it on the ground, before continuing her adventure." for weapon in weapons]
df['Prompt_Drop'] = sentences

In [77]:
excel_db_filename = 'DnD.xlsx'
db_name = "lldm_withdiscard.db"
lldm_assistant = LLDM_Assistant(OPENAI_API_KEY, db_name, excel_db_filename)

Database lldm.db formed.
CHARACTER_SHEET
INVENTORY
WORLD_ITEMS
SETTINGS
NPCS
TREASURES
MONSTERS
PLOT
LOGS


In [84]:
# Harder Mode with discard
# pick-up only
# do i need to initiate the narrator every round?

listIdxEval = []
for j in range(numRound):
    lldm_assistant = LLDM_Assistant(OPENAI_API_KEY, db_name)
    failed_count = 0
    evaluationHowManyInInventory = []
    evaluationHOLD_yes = []
    evaluationHOLD_no = []
    evaluationPickedMostRecent = []
    evaluationDroppedMostRecent = []
    evaluationDROP_no = []
    evaluationDROP_yes = []
    evaluationDROPFirst_yes = []
    
    print(f"Evaluation Round {j} Starts...")
    for i in range(numIter):
        print(f"Iteration {i} Starts...")
        if (i%6 == 0) | (i%6 == 4):
            # pick up an item
            idxPickup = i
            listIdxEval.append(idxPickup)
            tmp = lldm_assistant.narrator_chat(df['Prompt_Pickup'][idxPickup])
            response = tmp[0]['content']
            while (response is None) & (failed_count < maxFailed):
                tmp = lldm_assistant.narrator_chat(df['Prompt_Pickup'][idxPickup])
                response = tmp[0]['content']
                failed_count+=1
                print(f"failed count: {failed_count}")
            display(lldm_assistant.get_inventory_snapshot())
        elif (i%6 == 2):
            # drop an item: first picked that is not dropped in the inventory
            idxDrop = listIdxEval[int(np.floor(i/6))]
            tmp = lldm_assistant.narrator_chat(df['Prompt_Drop'][idxDrop])
            response = tmp[0]['content']
            while (response is None) & (failed_count < maxFailed):
                tmp = lldm_assistant.narrator_chat(df['Prompt_Drop'][idxDrop])
                response = tmp[0]['content']
                failed_count+=1
                print(f"failed count: {failed_count}")
            display(lldm_assistant.get_inventory_snapshot())
        elif i%6 == 5: 
            # question 1: how many
            tmp = lldm_assistant.narrator_chat("How many weapon or armor does Elara have?")
            response = tmp[0]['content']
            while (response is None) & (failed_count < maxFailed):
                tmp = lldm_assistant.narrator_chat("How many weapon or armor does Elara have?")
                response = tmp[0]['content']
                failed_count+=1
                print(f"failed count: {failed_count}")
            evaluationHowManyInInventory.append([i, response])
            time.sleep(1)
            
            # question 2: Do I have? Yes, not dropped
            idxEval = listIdxEval[-1:]
            weaponEval = df['Weapon Name'][idxEval]
            tmp = lldm_assistant.narrator_chat("Do I have " + weaponEval + " in inventory?")
            response = tmp[0]['content']
            while (response is None) & (failed_count < maxFailed):
                tmp = lldm_assistant.narrator_chat("Do I have " + weaponEval + " in inventory?")
                response = tmp[0]['content']
                failed_count+=1
                print(f"failed count: {failed_count}")
            evaluationHOLD_yes.append([i, weaponEval, response])
            time.sleep(1)
            
             # question 3: Do I have? No, dropped
            idxEval = idxDrop
            weaponEval = df['Weapon Name'][idxEval]
            tmp = lldm_assistant.narrator_chat("Do I have " + weaponEval + " in inventory?")
            response = tmp[0]['content']
            while (response is None) & (failed_count < maxFailed):
                tmp = lldm_assistant.narrator_chat("Do I have " + weaponEval + " in inventory?")
                response = tmp[0]['content']
                failed_count+=1
                print(f"failed count: {failed_count}")
            evaluationHOLD_no.append([i, weaponEval, response])
            time.sleep(1)
                      
            # question 4: last pickup
            idxEval = listIdxEval[-1:]
            tmp = lldm_assistant.narrator_chat("What was the last weapon or armor I obtained?")
            weaponEval = df['Weapon Name'][idxEval]
            response = tmp[0]['content']
            while (response is None) & (failed_count < maxFailed):
                tmp = lldm_assistant.narrator_chat("What was the last weapon or armor I obtained?")
                response = tmp[0]['content']
                failed_count+=1
                print(f"failed count: {failed_count}")
            evaluationPickedMostRecent.append([i, weaponEval, response])
            
            # question 5: last discard
            idxEval = idxDrop
            tmp = lldm_assistant.narrator_chat("What was the last weapon or armor I discarded?")
            weaponEval = df['Weapon Name'][idxEval]
            response = tmp[0]['content']
            while (response is None) & (failed_count < maxFailed):
                tmp = lldm_assistant.narrator_chat("What was the last weapon or armor I discarded?")
                response = tmp[0]['content']
                failed_count+=1
                print(f"failed count: {failed_count}")
            evaluationDroppedMostRecent.append([i, weaponEval, response])
            
            
            # question 6: Dropped? No, not dropped
            idxEval = listIdxEval[-1:]
            weaponEval = df['Weapon Name'][idxEval]
            tmp = lldm_assistant.narrator_chat("Have I discarded " + weaponEval + " from inventory?")
            response = tmp[0]['content']
            while (response is None) & (failed_count < maxFailed):
                tmp = lldm_assistant.narrator_chat("Have I discarded " + weaponEval + " from inventory?")
                response = tmp[0]['content']
                failed_count+=1
                print(f"failed count: {failed_count}")
            evaluationDROP_no.append([i, weaponEval, response])
            time.sleep(1)
            
            # question 7: Dropped? Yes, dropped
            idxEval = idxDrop
            weaponEval = df['Weapon Name'][idxEval]
            tmp = lldm_assistant.narrator_chat("Have I discarded " + weaponEval + " from inventory?")
            response = tmp[0]['content']
            while (response is None) & (failed_count < maxFailed):
                tmp = lldm_assistant.narrator_chat("Have I discarded " + weaponEval + " from inventory?")
                response = tmp[0]['content']
                failed_count+=1
                print(f"failed count: {failed_count}")
            evaluationDROP_yes.append([i, weaponEval, response])
            time.sleep(1)
            
            # question 8: Dropped? Yes, dropped (always ask the first dropped)
            idxEval = listIdxEval[0]
            weaponEval = df['Weapon Name'][idxEval]
            tmp = lldm_assistant.narrator_chat("Have I discarded " + weaponEval + " from inventory?")
            response = tmp[0]['content']
            while (response is None) & (failed_count < maxFailed):
                tmp = lldm_assistant.narrator_chat("Have I discarded " + weaponEval + " from inventory?")
                response = tmp[0]['content']
                failed_count+=1
                print(f"failed count: {failed_count}")
            evaluationDROPFirst_yes.append([i, weaponEval, response])
            time.sleep(1)
            
        else:
            # story telling prompt
            tmp = lldm_assistant.narrator_chat("Elara feels tired and decides to find a place to go to sleep. The night is peaceful and quiet. She wakes up the next morning and continues her adventure.")
            response = tmp[0]['content']
            while (response is None) & (failed_count < maxFailed):
                tmp = lldm_assistant.narrator_chat("Elara feels tired and decides to find a place to go to sleep. The night is peaceful and quiet. She wakes up the next morning and continues her adventure.")
                response = tmp[0]['content']
                failed_count+=1
                print(f"failed count: {failed_count}") 
        
        print(tmp[1]['content'].strip())
        print(response)
        time.sleep(10)      
        df_eval1 = pd.DataFrame(evaluationHowManyInInventory, columns=['iterations', 'answer'])
        df_eval2 = pd.DataFrame(evaluationHOLD_yes, columns=['iterations', 'item', 'answer'])
        df_eval3 = pd.DataFrame(evaluationHOLD_no, columns=['iterations', 'item', 'answer'])
        df_eval4 = pd.DataFrame(evaluationPickedMostRecent, columns=['iterations', 'item', 'answer'])
        df_eval5 = pd.DataFrame(evaluationDroppedMostRecent, columns=['iterations', 'item', 'answer'])
        df_eval6 = pd.DataFrame(evaluationDROP_no, columns=['iterations', 'item', 'answer'])
        df_eval7 = pd.DataFrame(evaluationDROP_yes, columns=['iterations', 'item', 'answer'])
        df_eval8 = pd.DataFrame(evaluationDROPFirst_yes, columns=['iterations', 'item', 'answer'])

        df_eval1.to_excel(f"{current_directory}/evaluation_llm4llm/HM_discard/HMDiscard_HowMany_{j}.xlsx")
        df_eval2.to_excel(f"{current_directory}/evaluation_llm4llm/HM_discard/HMDiscard_HoldYes_{j}.xlsx")
        df_eval3.to_excel(f"{current_directory}/evaluation_llm4llm/HM_discard/HMDiscard_HoldNo_{j}.xlsx")
        df_eval4.to_excel(f"{current_directory}/evaluation_llm4llm/HM_discard/HMDiscard_LastPicked_{j}.xlsx")
        df_eval5.to_excel(f"{current_directory}/evaluation_llm4llm/HM_discard/HMDiscard_LastDropped_{j}.xlsx")
        df_eval6.to_excel(f"{current_directory}/evaluation_llm4llm/HM_discard/HMDiscard_DropNo_{j}.xlsx")
        df_eval7.to_excel(f"{current_directory}/evaluation_llm4llm/HM_discard/HMDiscard_DropYes_{j}.xlsx")
        df_eval8.to_excel(f"{current_directory}/evaluation_llm4llm/HM_discard/HMDiscard_DropFirstYes_{j}.xlsx")
        
    time.sleep(10)
    display(lldm_assistant.get_inventory_snapshot())
        
        
    # delete the threads at the end of each round
    #client.beta.threads.delete(thread_narrator.id)
    #print(f"Evaluation Round {j} Threads Deleted.")
    #create_narrator()                  

Database lldm.db connected.
Evaluation Round 0 Starts...
Iteration 0 Starts...
requires_action
0 1


ValueError: Unknown function: get_obtained_item